In [1]:
import os

import pandas as pd
import math
import random as rd
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import pickle
import dlib

from deepface import DeepFace
from deepface.basemodels import VGGFace, OpenFace, Facenet, FbDeepFace, DeepID
from deepface.extendedmodels import Age, Gender, Race, Emotion
#from deepface.modules import verification
from deepface.commons import functions, distance as dst

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Dropout,LSTM,\
                            TimeDistributed,Flatten,Dense,Bidirectional,ConvLSTM2D,MaxPooling3D,AveragePooling2D,Lambda,\
                            Activation,BatchNormalization

from tensorflow import keras
tf.test.gpu_device_name()

2024-08-07 20:07:45.719066: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


''

In [2]:
seed_constant = 27
np.random.seed(seed_constant)
rd.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [3]:
DIR = '/var/scratch/mpa326/'

In [4]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Video Features/video_train_50m_50f', 'rb')     
df = pickle.load(dbfile)

y_train_50m_50f = df['labels']
gender_train_50m_50f = df['genders']
X_train_50m_50f=df['features']
race_train_50m_50f = df['races']
ethnicity_train_50m_50f = df['ethnicity']
paths_train_50m_50f = df['video_files_paths']

In [5]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Video Features/video_train_40m_60f', 'rb')     
df = pickle.load(dbfile)

y_train_40m_60f = df['labels']
gender_train_40m_60f = df['genders']
X_train_40m_60f=df['features']
race_train_40m_60f = df['races']
ethnicity_train_40m_60f = df['ethnicity']
paths_train_40m_60f = df['video_files_paths']

In [6]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Video Features/video_train_60m_40f', 'rb')     
df = pickle.load(dbfile)

y_train_60m_40f = df['labels']
gender_train_60m_40f = df['genders']
X_train_60m_40f=df['features']
race_train_60m_40f = df['races']
ethnicity_train_60m_40f = df['ethnicity']
paths_train_60m_40f = df['video_files_paths']

In [7]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Video Features/video_test_common', 'rb')     
df = pickle.load(dbfile)

y_test = df['labels']
gender_test = df['genders']
X_test=df['features']
race_test = df['races']
ethnicity_test = df['ethnicity']
paths_test = df['video_files_paths']

In [8]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Audio Features/audio_train_50m_50f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_50m_50f=df['features']

In [9]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Audio Features/audio_train_40m_60f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_40m_60f=df['features']

In [10]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Audio Features/audio_train_60m_40f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_60m_40f=df['features']

In [11]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Audio Features/audio_test_common', 'rb')     
df = pickle.load(dbfile)
X_test_aud=df['features']

In [12]:
from sklearn.utils import shuffle
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, race_train_50m_50f, ethnicity_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, race_train_50m_50f, ethnicity_train_50m_50f, random_state=27)
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, race_train_50m_50f, ethnicity_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, race_train_50m_50f, ethnicity_train_50m_50f, random_state=7)
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, race_train_50m_50f, ethnicity_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, race_train_50m_50f, ethnicity_train_50m_50f, random_state=42)

In [13]:
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, race_train_40m_60f, ethnicity_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, race_train_40m_60f, ethnicity_train_40m_60f, random_state=27)
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, race_train_40m_60f, ethnicity_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, race_train_40m_60f, ethnicity_train_40m_60f, random_state=7)
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, race_train_40m_60f, ethnicity_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, race_train_40m_60f, ethnicity_train_40m_60f, random_state=42)

In [14]:
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, race_train_60m_40f, ethnicity_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, race_train_60m_40f, ethnicity_train_60m_40f, random_state=27)
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, race_train_60m_40f, ethnicity_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, race_train_60m_40f, ethnicity_train_60m_40f, random_state=7)
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, race_train_60m_40f, ethnicity_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, race_train_60m_40f, ethnicity_train_60m_40f, random_state=42)

In [15]:
X_test, X_test_aud, y_test, gender_test, race_test, ethnicity_test = shuffle(X_test, X_test_aud, y_test, gender_test, race_test, ethnicity_test, random_state=27)
X_test, X_test_aud, y_test, gender_test, race_test, ethnicity_test = shuffle(X_test, X_test_aud, y_test, gender_test, race_test, ethnicity_test, random_state=7)
X_test, X_test_aud, y_test, gender_test, race_test, ethnicity_test = shuffle(X_test, X_test_aud, y_test, gender_test, race_test, ethnicity_test, random_state=42)

In [16]:
from tensorflow.keras.models import Model
emotion=Emotion.loadModel()
for layer in emotion.layers:
    print(layer.name)
features_emotion=Model(inputs=emotion.input, outputs=emotion.get_layer('dense_1').output)

conv2d
max_pooling2d
conv2d_1
conv2d_2
average_pooling2d
conv2d_3
conv2d_4
average_pooling2d_1
flatten
dense
dropout
dense_1
dropout_1
dense_2


In [17]:
features_emotion.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 48, 48, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 44, 44, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2  (None, 20, 20, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 18, 18, 64)        36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 average_pooling2d (Average  (None, 7, 7, 64)          0         
 Pooling2D)                                                  

In [18]:
len(X_test)

1487

In [19]:
df_ = np.ones((4678,20,48,48))
for i in range(4678):
  for j in range(20):
    img=X_train_50m_50f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_50m_50f=df_
df_=[]

In [20]:
df_ = np.ones((4678,20,48,48))
for i in range(4678):
  for j in range(20):
    img=X_train_40m_60f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_40m_60f=df_
df_=[]

In [21]:
df_ = np.ones((4677,20,48,48))
for i in range(4677):
  for j in range(20):
    img=X_train_60m_40f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_60m_40f=df_
df_=[]

In [22]:
df_ = np.ones((1487,20,48,48))
for i in range(1487):
  for j in range(20):
    img=X_test[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_test=df_
df_=[]

In [23]:
features_emotion.trainable=False

In [24]:
#Spatial transformer network (STN)

In [24]:
class Localization(tf.keras.layers.Layer):
    def __init__(self):
        super(Localization, self).__init__()
        self.pool1 = tf.keras.layers.MaxPool2D()
        self.conv1 = tf.keras.layers.Conv2D(20, [5, 5], activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D()
        self.conv2 = tf.keras.layers.Conv2D(20, [5, 5], activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(20, activation='relu')
        self.fc2 = tf.keras.layers.Dense(6, activation=None, bias_initializer=tf.keras.initializers.constant([1.0, 0.0, 0.0, 0.0, 1.0, 0.0]), kernel_initializer='zeros')

    def build(self, input_shape):
        print("Building Localization Network with input shape:", input_shape)

    def compute_output_shape(self, input_shape):
        return [None, 2, 3]

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        theta = self.fc2(x)
        theta = tf.keras.layers.Reshape((2, 3))(theta)
        return theta

In [25]:
class BilinearInterpolation(tf.keras.layers.Layer):
    def __init__(self, height=40, width=40):
        super(BilinearInterpolation, self).__init__()
        self.height = height
        self.width = width

    def compute_output_shape(self, input_shape):
        return [None, self.height, self.width, 1]

    def get_config(self):
        return {
            'height': self.height,
            'width': self.width,
        }
    
    def build(self, input_shape):
        print("Building Bilinear Interpolation Layer with input shape:", input_shape)

    def advance_indexing(self, inputs, x, y):
        '''
        Numpy like advance indexing is not supported in tensorflow, hence, this function is a hack around the same method
        '''        
        shape = tf.shape(inputs)
        batch_size, _, _ = shape[0], shape[1], shape[2]
        
        batch_idx = tf.range(0, batch_size)
        batch_idx = tf.reshape(batch_idx, (batch_size, 1, 1))
        b = tf.tile(batch_idx, (1, self.height, self.width))
        indices = tf.stack([b, y, x], 3)
        return tf.gather_nd(inputs, indices)

    def call(self, inputs):
        images, theta = inputs
        homogenous_coordinates = self.grid_generator(batch=tf.shape(images)[0])
        return self.interpolate(images, homogenous_coordinates, theta)

    def grid_generator(self, batch):
        x = tf.linspace(-1, 1, self.width)
        y = tf.linspace(-1, 1, self.height)
            
        xx, yy = tf.meshgrid(x, y)
        xx = tf.reshape(xx, (-1,))
        yy = tf.reshape(yy, (-1,))
        homogenous_coordinates = tf.stack([xx, yy, tf.ones_like(xx)])
        homogenous_coordinates = tf.expand_dims(homogenous_coordinates, axis=0)
        homogenous_coordinates = tf.tile(homogenous_coordinates, [batch, 1, 1])
        homogenous_coordinates = tf.cast(homogenous_coordinates, dtype=tf.float32)
        return homogenous_coordinates
    
    def interpolate(self, images, homogenous_coordinates, theta):

        with tf.name_scope("Transformation"):
            transformed = tf.matmul(theta, homogenous_coordinates)
            transformed = tf.transpose(transformed, perm=[0, 2, 1])
            transformed = tf.reshape(transformed, [-1, self.height, self.width, 2])
                
            x_transformed = transformed[:, :, :, 0]
            y_transformed = transformed[:, :, :, 1]
                
            x = ((x_transformed + 1.) * tf.cast(self.width, dtype=tf.float32)) * 0.5
            y = ((y_transformed + 1.) * tf.cast(self.height, dtype=tf.float32)) * 0.5

        with tf.name_scope("VariableCasting"):
            x0 = tf.cast(tf.math.floor(x), dtype=tf.int32)
            x1 = x0 + 1
            y0 = tf.cast(tf.math.floor(y), dtype=tf.int32)
            y1 = y0 + 1

            x0 = tf.clip_by_value(x0, 0, self.width-1)
            x1 = tf.clip_by_value(x1, 0, self.width-1)
            y0 = tf.clip_by_value(y0, 0, self.height-1)
            y1 = tf.clip_by_value(y1, 0, self.height-1)
            x = tf.clip_by_value(x, 0, tf.cast(self.width, dtype=tf.float32)-1.0)
            y = tf.clip_by_value(y, 0, tf.cast(self.height, dtype=tf.float32)-1)

        with tf.name_scope("AdvanceIndexing"):
            Ia = self.advance_indexing(images, x0, y0)
            Ib = self.advance_indexing(images, x0, y1)
            Ic = self.advance_indexing(images, x1, y0)
            Id = self.advance_indexing(images, x1, y1)

        with tf.name_scope("Interpolation"):
            x0 = tf.cast(x0, dtype=tf.float32)
            x1 = tf.cast(x1, dtype=tf.float32)
            y0 = tf.cast(y0, dtype=tf.float32)
            y1 = tf.cast(y1, dtype=tf.float32)
                            
            wa = (x1-x) * (y1-y)
            wb = (x1-x) * (y-y0)
            wc = (x-x0) * (y1-y)
            wd = (x-x0) * (y-y0)

            wa = tf.expand_dims(wa, axis=3)
            wb = tf.expand_dims(wb, axis=3)
            wc = tf.expand_dims(wc, axis=3)
            wd = tf.expand_dims(wd, axis=3)
                        
        return tf.math.add_n([wa*Ia + wb*Ib + wc*Ic + wd*Id])

In [26]:
emotions_en = ['ANG', 'DIS', 'FEA', 'HAP', 'NEU', 'SAD']

In [27]:
def create__LRCN_with_STN() :
  image = tf.keras.layers.Input(shape=(20,48,48,1))
  theta = TimeDistributed(Localization())(image)
  x = TimeDistributed(BilinearInterpolation(height=48, width=48))([image, theta])
  features=TimeDistributed(features_emotion)(x)
  lstm=Bidirectional(LSTM(200, activation='tanh',input_shape=(20, 1024),dropout=.3))(features)
  out=Dense(len(emotions_en), activation = 'softmax')(lstm)

  return tf.keras.models.Model(inputs=image, outputs=out)


In [28]:
create__LRCN_with_STN().summary()

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 20, 48, 48, 1)]      0         []                            
                                                                                                  
 time_distributed (TimeDist  (None, 20, 2, 3)             43086     ['input_1[0][0]']             
 ributed)                                                                                         
                                                                                                  
 time_distributed_1 (TimeDi  (None, 20, 48, 48, 1)        0         ['input_1[0][0]',             
 stributed)                        

In [29]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [30]:
from sklearn.model_selection import StratifiedKFold

seed = 7
kfold = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)

In [31]:
unique_values, counts = np.unique(gender_train_50m_50f, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"Value {value} appears {count} times")


Value Female appears 2338 times
Value Male appears 2340 times


In [32]:
unique_values, counts = np.unique(race_train_50m_50f, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"Value {value} appears {count} times")


Value African American appears 1139 times
Value Asian appears 346 times
Value Caucasian appears 3140 times
Value Unknown appears 53 times


In [33]:
unique_values, counts = np.unique(gender_train_60m_40f, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"Value {value} appears {count} times")


Value Female appears 1870 times
Value Male appears 2807 times


In [34]:
unique_values, counts = np.unique(race_train_60m_40f, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"Value {value} appears {count} times")


Value African American appears 1176 times
Value Asian appears 376 times
Value Caucasian appears 3085 times
Value Unknown appears 40 times


In [36]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = os.path.abspath(DIR)+'/cremad_60m40f_video_model/'
fold_var = 1

for train_idx, val_idx in kfold.split(X_train_60m_40f, y_train_60m_40f):
  # with tpu_strategy.scope():
  model=create__LRCN_with_STN() 
  model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
  early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)
  checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
        monitor='val_accuracy', 
        save_best_only=True, mode='max')

  LRCN_model_training_history = model.fit(        x = X_train_60m_40f[train_idx],
                                                  y = y_train_60m_40f[train_idx],
                                                  validation_data=(X_train_60m_40f[val_idx],y_train_60m_40f[val_idx]),
                                                  epochs = 80,
                                                  batch_size = 32,
                                                  shuffle = True,
                                                  callbacks = [checkpoint_cb,early_stopping_callback])

  model.load_weights(save_dir+"model_"+str(fold_var)+".h5")
	
  results = model.evaluate(X_test, y_test)
  results = dict(zip(model.metrics_names,results))
	
  VALIDATION_ACCURACY.append(results['accuracy'])
  VALIDATION_LOSS.append(results['loss'])
	
  tf.keras.backend.clear_session()
	
  fold_var += 1

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))
Epoch 1/80
117/117 [==============================] - 88s 707ms/step - loss: 1.4053 - accuracy: 0.4362 - val_loss: 1.3996 - val_accuracy: 0.4295
Epoch 2/80
117/117 [==============================] - 81s 694ms/step - loss: 1.2630 - accuracy: 0.4975 - val_loss: 1.2819 - val_accuracy: 0.4872
Epoch 3/80
117/117 [==============================] - 81s 693ms/step - loss: 1.1728 - accuracy: 0.5394 - val_loss: 1.2129 - val_accuracy: 0.5353
Epoch 4/80
117/117 [==============================] - 82s 699ms/step - loss: 1.0991 - accuracy: 0.5624 - val_loss: 1.1476 - val_accuracy: 0.5385
Epoch 5/80
117/117 [==============================] - 83s 713ms/step - loss: 1.0287 - accuracy: 0.5929 - val_loss: 1.1055 - val_accuracy: 0.5630
Epoch 6/80
117/117 [==============================] - 83s 709ms/step - loss: 0.9667 - accuracy: 0.6236 - val_loss: 1.10

In [35]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = os.path.abspath(DIR)+'/cremad_40m60f_video_model/'
fold_var = 1

for train_idx, val_idx in kfold.split(X_train_40m_60f, y_train_40m_60f):
  # with tpu_strategy.scope():
  model=create__LRCN_with_STN() 
  model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
  early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)
  checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
        monitor='val_accuracy', 
        save_best_only=True, mode='max')

  LRCN_model_training_history = model.fit(        x = X_train_40m_60f[train_idx],
                                                  y = y_train_40m_60f[train_idx],
                                                  validation_data=(X_train_40m_60f[val_idx],y_train_40m_60f[val_idx]),
                                                  epochs = 80,
                                                  batch_size = 32,
                                                  shuffle = True,
                                                  callbacks = [checkpoint_cb,early_stopping_callback])

  model.load_weights(save_dir+"model_"+str(fold_var)+".h5")
	
  results = model.evaluate(X_test, y_test)
  results = dict(zip(model.metrics_names,results))
	
  VALIDATION_ACCURACY.append(results['accuracy'])
  VALIDATION_LOSS.append(results['loss'])
	
  tf.keras.backend.clear_session()
	
  fold_var += 1

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))
Epoch 1/80
117/117 [==============================] - 93s 759ms/step - loss: 1.3641 - accuracy: 0.4487 - val_loss: 1.2290 - val_accuracy: 0.5139
Epoch 2/80
117/117 [==============================] - 90s 769ms/step - loss: 1.1944 - accuracy: 0.5323 - val_loss: 1.1768 - val_accuracy: 0.5502
Epoch 3/80
117/117 [==============================] - 89s 765ms/step - loss: 1.0862 - accuracy: 0.5730 - val_loss: 1.1709 - val_accuracy: 0.5310
Epoch 4/80
117/117 [==============================] - 89s 762ms/step - loss: 1.0194 - accuracy: 0.6021 - val_loss: 1.1992 - val_accuracy: 0.5246
Epoch 5/80
117/117 [==============================] - 89s 764ms/step - loss: 0.9552 - accuracy: 0.6264 - val_loss: 1.1141 - val_accuracy: 0.5759
Epoch 6/80
117/117 [==============================] - 89s 759ms/step - loss: 0.8887 - accuracy: 0.6574 - val_loss: 1.11

In [ ]:
#Audio

In [19]:
X_train_aud_50m_50f.shape

(4678, 128, 128, 3)

In [21]:
#Normalize values
X_train_aud_50m_50f = X_train_aud_50m_50f/255
X_train_aud_40m_60f = X_train_aud_40m_60f/255
X_train_aud_60m_40f = X_train_aud_60m_40f/255
X_test_aud = X_test_aud/255

In [22]:
def model_alexnet():
  AlexNet = Sequential()

  #1st Convolutional Layer
  AlexNet.add(Conv2D(filters=96, input_shape=(128,128,3), kernel_size=(11,11), strides=(4,4), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #2nd Convolutional Layer
  AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #3rd Convolutional Layer
  AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #4th Convolutional Layer
  AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #5th Convolutional Layer
  AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #Passing it to a Fully Connected layer
  AlexNet.add(Flatten())
  AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(Dropout(0.4))

  #2nd Fully Connected Layer
  AlexNet.add(Dense(1000))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  #Add Dropout
  AlexNet.add(Dropout(0.4))

  #Output Layer
  AlexNet.add(Dense(6))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('softmax'))

  return AlexNet 

In [23]:
model_alexnet().summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 96)        34944     
                                                                 
 batch_normalization (Batch  (None, 32, 32, 96)        384       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 32, 32, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 32, 32, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 256)       614656    
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 256)       1

In [ ]:
VALIDATION_ACCURACY_SPEECH = []
VALIDATION_LOSS_SPEECH= []

save_dir = os.path.abspath(DIR)+'/cremad_60m40f_audio_model/'
fold_var = 1

for train_idx, val_idx in kfold.split(X_train_aud_60m_40f, y_train_60m_40f):
  # with tpu_strategy.scope()
  model=model_alexnet() 
  model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
  early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)
  checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
        monitor='val_accuracy', 
        save_best_only=True, mode='max')

  LRCN_model_training_history = model.fit(        x = X_train_aud_60m_40f[train_idx],
                                                  y = y_train_60m_40f[train_idx],
                                                  validation_data=(X_train_aud_60m_40f[val_idx], y_train_60m_40f[val_idx]),
                                                  epochs = 80,
                                                  batch_size = 32,
                                                  shuffle = True,
                                                  callbacks = [checkpoint_cb,early_stopping_callback])

  model.load_weights(save_dir+"model_"+str(fold_var)+".h5")
	
  results = model.evaluate(X_test_aud, y_test)
  results = dict(zip(model.metrics_names,results))
	
  VALIDATION_ACCURACY_SPEECH.append(results['accuracy'])
  VALIDATION_LOSS_SPEECH.append(results['loss'])
	
  tf.keras.backend.clear_session()
	
  fold_var += 1

Epoch 1/80
117/117 [==============================] - 821s 7s/step - loss: 1.5011 - accuracy: 0.4007 - val_loss: 1.9553 - val_accuracy: 0.3173
Epoch 2/80
117/117 [==============================] - 784s 7s/step - loss: 1.3859 - accuracy: 0.4509 - val_loss: 2.0464 - val_accuracy: 0.2479
Epoch 3/80
117/117 [==============================] - 829s 7s/step - loss: 1.3324 - accuracy: 0.4868 - val_loss: 3.6556 - val_accuracy: 0.3472
Epoch 4/80
117/117 [==============================] - 827s 7s/step - loss: 1.3035 - accuracy: 0.4940 - val_loss: 2.9851 - val_accuracy: 0.3483
Epoch 5/80
117/117 [==============================] - 909s 8s/step - loss: 1.2495 - accuracy: 0.5237 - val_loss: 2.9249 - val_accuracy: 0.3953
Epoch 6/80
117/117 [==============================] - 824s 7s/step - loss: 1.2114 - accuracy: 0.5319 - val_loss: 6.5184 - val_accuracy: 0.2041
Epoch 7/80
117/117 [==============================] - 833s 7s/step - loss: 1.1609 - accuracy: 0.5600 - val_loss: 2.8540 - val_accuracy: 0.2489